In [115]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Dropout, Flatten
input_shape = (150, 150, 3)

vgg = tf.keras.applications.vgg16.VGG16(include_top = False, weights='imagenet', input_shape=input_shape)

vgg.trainable = False
for layer in vgg.layers:
    layer.trainable = False
model = keras.models.Sequential(vgg.layers)
model.add(Flatten())
model.add(Dense(512, activation='relu', input_dim=ins))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)      

In [116]:

for layer in model.layers:
    print(layer.name, layer.trainable)

block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
flatten_5 True
dense_9 True
dropout_6 True
dense_10 True
dropout_7 True
dense_11 True


In [1]:
import glob 
import numpy as np
import os 
import shutil

In [127]:
np.random.seed(42)
base = 'garbage/dogs vs cats/'
files = glob.glob(base+'train/*')

catf=[fn for fn in files if 'cat' in fn.split('\\')[1]]
dogf=[fn for fn in files if 'dog' in fn.split('\\')[1]]
print(len(catf), len(dogf))

cattr = np.random.choice(catf, size=1500, replace=False)
dogtr = np.random.choice(dogf, size=1500, replace=False)

catf = list(set(catf)-set(cattr))
dogf = list(set(dogf)-set(dogtr))

catv = np.random.choice(catf, size=500, replace=False)
dogv = np.random.choice(dogf, size=500, replace=False)

catf = list(set(catf)-set(catv))
dogf = list(set(dogf)-set(dogv))

catte = np.random.choice(catf, size=500, replace=False)
dogte = np.random.choice(dogf, size=500, replace=False)

print(cattr.shape,catte.shape, catv.shape)
print(dogtr.shape,dogte.shape, dogv.shape)

trf = np.concatenate([cattr, dogtr])
vf = np.concatenate([catv, dogv])
tef = np.concatenate([catte, dogte])

print(trf.shape, vf.shape, tef.shape)

import cv2 as cv

imgtr = [cv.resize(cv.imread(img), (150,150)) for img in trf]
labtr = [fn.split('\\')[1].split('.')[0].strip() for fn in trf]

imgtr = np.asarray(imgtr)
imgtr.shape

imgv = [cv.resize(cv.imread(img), (150,150)) for img in vf]
labv = [fn.split('\\')[1].split('.')[0].strip() for fn in vf]

imgv = np.asarray(imgv)
imgv.shape

imgtrs = imgtr.astype('float32')
imgvs = imgv.astype('float32')

imgtrs /= 255
imgvs /= 255

print(imgtrs[0].shape)

labtre = []

for i in labtr:
    if i == 'cat':
        labtre.append(0)
#         print(0)
    else:
        labtre.append(1)
#         print(1)

labve = []

for i in labv:
    if i == 'cat':
        labve.append(0)
#         print(0)
    else:
        labve.append(1)
#         print(1)

12500 12500
(1500,) (500,) (500,)
(1500,) (500,) (500,)
(3000,) (1000,) (1000,)
(150, 150, 3)


In [126]:
his = model.fit(x=imgtrs, y=labtre,
               validation_data=(imgvs, labve),
               batch_size=30,
               epochs=30)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})